In [2]:
import pandas as pd 
import re
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_colwidth',100)
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

df = pd.read_csv('test42_cleaned.csv')
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] File b'test42_cleaned.csv' does not exist: b'test42_cleaned.csv'

In [ ]:
df = df[(df['JobDesc']!='[]')]
df

In [ ]:
def clean_text(text):
    text.split('\n')
    text="".join([char.lower() for char in text if char not in string.punctuation])
    tokens = re.split("\W+",text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return str(text)

df['JobCleaned'] = df['JobDesc'].apply(lambda x : clean_text(x))
tfidf_vect = TfidfVectorizer()
X_tfidf = tfidf_vect.fit_transform(df['JobCleaned'])
X_features = pd.concat([df['CompanyName'], df['JobTitle'],pd.DataFrame(X_tfidf.toarray())],axis=1)
X_features=X_features.dropna()
X_features

In [ ]:
#Feature Engineering

def isJunior(title):
    junior_names = ['junior','Junior','Entry','entry','graduate','Graduate','Intern','intern','internship']  
    for word in title.split():
        if word in junior_names:
            return True
        else:
            return False

        
X_features['isJunior'] = X_features['JobTitle'].apply(lambda x : isJunior(x))

In [ ]:
X_features.to_csv('testdata.csv', index=False, header=True)

In [14]:
train = pd.read_csv('testdata.csv')
train.drop_duplicates(subset=['CompanyName','JobTitle'], keep="last", inplace=True)
train

,CompanyName,JobTitle,Applied,0,1,2,3,4,5,6,...,2089,2090,2091,2092,2093,2094,2095,2096,2097,isJunior
0,Verisk Financial,Associate Analyst - SQL,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.000000,0.0,0.127067,0,0,0.000000,0.000000,0.000000,False
3,Commonwealth Bank,Credit Risk Analyst - Expression of Interest,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
5,Commonwealth Bank,Program Business Analyst,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.041032,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
6,Qantas Airways Limited,Marketing Insights Analyst,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
8,QBE,Catastrophe Modelling Analyst,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
9,Woolworths Supermarkets,Business Analyst,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
10,Woolworths Supermarkets,Procurement Analyst (12 month secondment),No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.027124,0.0,0.000000,0,0,0.000000,0.000000,0.077219,False
11,Commonwealth Bank,Business Risk Analyst,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
12,Telstra Health,Business Analyst - 6 month FTC,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.026233,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
13,Veeva Systems,Junior Data Analyst (Contract),No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.024684,0.0,0.000000,0,0,0.000000,0.000000,0.000000,True


In [15]:
def encode_categoricals(df, categoricals):
    for c in categoricals:
        df[c] = df[c].astype("category").cat.codes
    return df

categoricals = [
    'CompanyName',
    'JobTitle'
]

train_df = encode_categoricals(train,categoricals)
train

,CompanyName,JobTitle,Applied,0,1,2,3,4,5,6,...,2089,2090,2091,2092,2093,2094,2095,2096,2097,isJunior
0,24,2,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.000000,0.0,0.127067,0,0,0.000000,0.000000,0.000000,False
3,4,10,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
5,4,25,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.041032,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
6,15,21,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
8,14,6,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
9,25,3,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
10,25,24,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.027124,0.0,0.000000,0,0,0.000000,0.000000,0.077219,False
11,4,5,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
12,21,4,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.026233,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False
13,23,19,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.024684,0.0,0.000000,0,0,0.000000,0.000000,0.000000,True


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
X_train = train_df.drop('Applied',axis=1)
response = train_df['Applied']
cross_val_score(rf,X_train,train['Applied'], cv=k_fold,scoring='accuracy', n_jobs=-1)

array([0.625     , 0.71428571, 0.57142857, 0.57142857, 0.71428571])

In [17]:
rf_model =rf.fit(X_train,response)

df_importance = pd.DataFrame({
    "Feature":X_train.columns,
    "Importance":rf_model.feature_importances_
})
df_importance = df_importance.sort_values(by="Importance", ascending=False)
print(df_importance)

C:\Users\Eddie\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


          Feature  Importance
1        JobTitle    0.095785
0     CompanyName    0.082485
1871         1869    0.072827
523           521    0.072808
1429         1427    0.072727
...           ...         ...
703           701    0.000000
702           700    0.000000
701           699    0.000000
699           697    0.000000
2100     isJunior    0.000000

[2101 rows x 2 columns]


In [18]:
train['Prediction'] = rf_model.predict(X_train)
filtered_jobs = train[train['Prediction']=='Yes'][['CompanyName','JobTitle']]
filtered_jobs


,CompanyName,JobTitle
0,24,2
5,4,25
27,4,18
31,3,11
34,6,0
40,9,3
46,0,13
47,10,9


In [25]:
filtered_jobs.index

Int64Index([0, 5, 27, 31, 34, 40, 46, 47], dtype='int64')

In [41]:
train = pd.read_csv('testdata.csv')
test = []
for i in filtered_jobs.index:
    test.append(str(train[train.index==i][['CompanyName','JobTitle']]).strip().split())

for job in test:
    for word in job[:3]:
        

[['CompanyName',
  'JobTitle',
  '0',
  'Verisk',
  'Financial',
  'Associate',
  'Analyst',
  '-',
  'SQL'],
 ['CompanyName',
  'JobTitle',
  '5',
  'Commonwealth',
  'Bank',
  'Program',
  'Business',
  'Analyst'],
 ['CompanyName',
  'JobTitle',
  '27',
  'Commonwealth',
  'Bank',
  'Insights',
  'Analyst',
  '-',
  'Expression',
  'of',
  'Interest'],
 ['CompanyName',
  'JobTitle',
  '31',
  'Allianz',
  'Customer',
  'Insights',
  'Analyst'],
 ['CompanyName',
  'JobTitle',
  '34',
  'Dimensional',
  'Fund',
  'Advisors',
  'Analyst,',
  'Investment',
  'Analytics',
  'and',
  'Data'],
 ['CompanyName', 'JobTitle', '40', 'Genworth', 'Business', 'Analyst'],
 ['CompanyName', 'JobTitle', '46', 'AMP', 'Limited', 'Data', 'Analyst'],
 ['CompanyName',
  'JobTitle',
  '47',
  'IBM',
  'Cloud',
  'delivery',
  'services',
  'support',
  '-',
  'Maximo']]

,CompanyName,JobTitle,Applied,0,1,2,3,4,5,6,...,2090,2091,2092,2093,2094,2095,2096,2097,isJunior,Prediction
0,24,2,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.0,0.127067,0,0,0.000000,0.000000,0.000000,False,Yes
3,4,10,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,No
5,4,25,Yes,0,0,0.000000,0,0.000000,0,0.000000,...,0.041032,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,Yes
6,15,21,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,No
8,14,6,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,No
9,25,3,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,No
10,25,24,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.027124,0.0,0.000000,0,0,0.000000,0.000000,0.077219,False,No
11,4,5,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.031085,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,No
12,21,4,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.026233,0.0,0.000000,0,0,0.000000,0.000000,0.000000,False,No
13,23,19,No,0,0,0.000000,0,0.000000,0,0.000000,...,0.024684,0.0,0.000000,0,0,0.000000,0.000000,0.000000,True,No
